# ResNet_from_scratch (blank)

- 빈칸을 주석, 치트시트, 쿡북을 활용해 채워보시기 바랍니다.
- 빈칸 옆 주석은 **역할과 의도** 중심으로 작성되었습니다.
- 아래 **실습 코드** 는 빈칸없이 제공됩니다.

> 사용법  
> 1) 위에서부터 내려오며 `____`만 채우세요.  
> 2) 각 섹션의 체크 테스트를 수행해보세요.  
> 3) 마지막 “실습: CIFAR-10 학습” 섹션을 실행해보세요(시간이 오래걸릴 수 있습니다.)


## 학습 목표

1. **Residual addition**(`out + identity`)이 왜 학습을 안정화하는지 설명할 수 있다.  
2. Shortcut의 **Option A(Identity + Zero-pad)** / **Option B(Projection)** 를 구분하고, 언제 필요한지(채널/stride 불일치) 판단할 수 있다.  
3. CIFAR-ResNet의 **stage 구조**와 “stage 시작에서만 downsample(stride=2)” 규칙을 코드로 구현할 수 있다.  
4. CIFAR 실험의 **depth = 6n + 2 규칙**을 코드로 계산하고 검증할 수 있다.  
5. 학습/평가 루프에서 `train() / eval() / no_grad()`의 역할을 코드 레벨에서 정확히 구분할 수 있다.


## Option A / Option B 용어 정리

CheatSheet 기준으로 **Option A는 “파라미터 없는 shortcut” 전체**를 의미합니다.

- **Option A (기본형 = Identity Shortcut)**  
  - 조건: `stride==1` **그리고** `in_channels == out_channels`  
  - 구현: `nn.Identity()`  
  - 의미: 입력을 **아무 변화 없이** 그대로 더할 수 있는 상황

- **Option A (응용형 = Zero-padding Shortcut, CIFAR 실험에서 사용)**  
  - 조건: stage 경계에서 `stride!=1` 이거나 `out_channels != in_channels`  
  - 구현: `ShortcutZeroPad`  
  - 의미: 학습 가능한 `W_s` 없이 **(H,W)와 채널 수만 맞춰서** `F(x)+x`를 유지  
  - ⚠️ 주의: “항상 쓰는 identity”가 아니라, **차원이 안 맞을 때만** 등장하는 Option A 입니다.

- **Option B (Projection Shortcut)**  
  - 조건: 차원 불일치가 있을 때 “성능/안정성”을 위해 학습 가능한 매핑을 허용  
  - 구현: `ShortcutProjection` (보통 1×1 conv)

따라서 이 노트북에서 **`ShortcutZeroPad` 셀 제목에 Option A가 붙어 있어도**,  
그 의미는 “Option A 중에서도 **차원 불일치(mismatch) 상황을 해결하는 응용형**”을 구현한다는 뜻입니다.


## (선택) 실행을 위한 설치

이미 환경에 PyTorch/torchvision이 있다면 건너뛰세요.


In [ ]:

# 필요 시만 실행
# !pip -q install torch torchvision


## 0) Residual Learning 핵심 ↔ `BasicBlockV1.forward`의 "add" 라인

- **핵심 수식**:  \( y = F(x) + x \)  
- 직관: “원본(x)을 통째로 다시 만들지 말고, 필요한 수정분(F(x))만 더한다.”


## 1) Import (모델 파트)


In [ ]:
from typing import List, Type
import torch
import torch.nn as nn


# =========================================================
# 1) Shortcut (Option A: zero-pad, Option B: projection)
# =========================================================


## 1) Shortcut Option A (Identity + Zero-padding) ↔ `ShortcutZeroPad.forward`


✅ 핵심: **Option A는 (shape가 맞으면) Identity, (안 맞으면) Zero-padding** 으로 생각하면 가장 덜 헷갈립니다.
Option A는 **파라미터를 늘리지 않고**(학습되는 W_s 없음) 차원만 맞춥니다.

- stride로 H,W가 줄면 shortcut도 **같이 줄여야** 더하기가 가능합니다.
- 채널이 늘면 `0` 채널을 붙여서 `(N, out_channels, H, W)`로 맞춥니다.


In [ ]:

class ShortcutZeroPad(nn.Module):
    """
    Option A (CIFAR): **학습 파라미터 없이** 차원만 맞추는 shortcut.
    - stage 경계에서 (H,W)가 줄어들거나 채널이 늘어날 때도 'F(x) + shortcut(x)'가 가능하도록
      shortcut 경로의 shape를 맞춥니다.
    """

    def __init__(self, in_channels: int, out_channels: int, stride: int):
        super().__init__()
        if out_channels < in_channels:
            raise ValueError(
                "ShortcutZeroPad only supports channel increase (out_channels >= in_channels). "
                "Use projection shortcut (1x1 conv) for channel decrease."
            )
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # NOTE (Option A 정리)
        # - stride==1이고 채널 변화가 없으면, 사실상 identity와 동일하게 동작합니다.
        # - 차원이 불일치할 때만 downsample/zero-pad를 수행합니다.

        # [CheatSheet §1 Option A(Zero-padding) | Step 1/4] (H,W) 맞추기:
        # 메인 경로가 stride로 해상도를 줄였다면, shortcut도 같은 비율로 줄여야 add가 가능합니다.
        if self.stride != 1:
            x = _[_,_,______,______]  # 설명: stride에 맞춰 (H,W)를 축소해 main path와 동일한 공간 크기로 맞춘다
            # 힌트: 쿡북 참조

        # [CheatSheet §1 Option A(Zero-padding) | Step 2/4] (C) 맞추기:
        # 출력 채널에 맞추기 위해 "추가로 필요한 채널 수"를 계산합니다.
        pad_channels = _________________ - ________________  # 설명: 출력 채널 수와 입력 채널 수의 차이(추가해야 하는 채널 개수)

        if pad_channels == 0:
            return x

        # [CheatSheet §1 Option A(Zero-padding) | Step 3/4] 0 텐서 만들기:
        # (N, 추가채널, H, W) 형태로 만들어 channel 축에 이어 붙일 준비를 합니다.
        zeros = torch.zeros(
            (x.size(0), ___________, x.size(2), x.size(3)),  # 설명: 두 번째 축이 "추가 채널" 크기
            device=x.device,
            dtype=x.dtype,
        )

        # [CheatSheet §1 Option A(Zero-padding) | Step 4/4] channel 축으로 concat
        return torch.cat([x, zeros], dim=__)  # 설명: 채널 축으로 이어 붙여 out_channels를 맞춘다


### ✅ Check 1: ShortcutZeroPad shape 테스트
(빈칸을 채운 뒤 실행)


In [ ]:
# ShortcutZeroPad: in=16 -> out=32, stride=2
sp = ShortcutZeroPad(in_channels=16, out_channels=32, stride=2)
x = torch.randn(4, 16, 32, 32)
y = sp(x)

expected = (4, 32, 16, 16)
actual = tuple(y.shape)

if actual != expected:
    raise ValueError(f"shape mismatch: expected {expected}, got {actual}")

print("[OK] ShortcutZeroPad shape:", actual)


## 2) Shortcut Option B (Projection) ↔ `ShortcutProjection`

Option B는 1×1 conv(+BN)로 입력을 **학습적으로** 변환해 차원을 맞춥니다.


In [ ]:

class ShortcutProjection(nn.Module):
    """
    Option B: 1×1 conv(+BN)로 shortcut 경로도 **학습 가능하게** 만들어 차원을 맞춥니다.
    - 차원 불일치가 생기는 지점에서 `W_s x` 형태로 변환한 뒤 더할 수 있게 합니다.
    """

    def __init__(self, in_channels: int, out_channels: int, stride: int):
        super().__init__()
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=1,
            stride=stride, 
            padding=0,
            bias=False,
        )
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return ____.__(____.______(__))  # 설명: 1x1 conv로 채널과 공간 크기를 맞춘 뒤 BN 적용

# =========================================================
# 2) Residual Block (ResNet v1: post-activation)
# =========================================================


### ✅ Check 2: ShortcutProjection shape 테스트
(빈칸을 채운 뒤 실행)


In [ ]:
sp = ShortcutProjection(in_channels=16, out_channels=32, stride=2)
x = torch.randn(4, 16, 32, 32)
y = sp(x)

expected = (4, 32, 16, 16)
actual = tuple(y.shape)

if actual != expected:
    raise ValueError(f"shape mismatch: expected {expected}, got {actual}")

print("[OK] ShortcutProjection shape:", actual)


## 3) BasicBlockV1 (ResNet v1: post-activation) ↔ `BasicBlockV1`

ResNet v1의 기본 블록(논문 Figure 2)은 아래 흐름입니다.

1) conv → bn → relu  
2) conv → bn  
3) add(identity)  
4) relu


**CheatSheet 대응 포인트**
- shortcut 선택 로직: Identity / Zero-pad / Projection (Option A/B)
- `out + identity`: Residual Addition (Eq.1)
- add 이후 ReLU: Post-activation (ResNet v1)


In [ ]:

class BasicBlockV1(nn.Module):
    """
    Figure 2 (left): conv-bn-relu -> conv-bn -> add -> relu  (ResNet v1)
    """
    expansion = 1  # BasicBlock은 채널 확장(expansion)이 없습니다.

    def __init__(self, in_channels: int, out_channels: int, stride: int, shortcut: str = "projection"):
        super().__init__()

        # 메인 경로(F): 3×3 conv 두 번 (CIFAR-ResNet 기본)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        # 두 번째 conv는 블록 내부에서 추가 downsample을 하지 않도록 stride=1 고정
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

        # [CheatSheet §1 Shortcut 연결 방식 | Step 9]
        # (1) shape가 같으면 가장 단순한 shortcut(항등)을 쓴다.
        if stride == 1 and in_channels == out_channels:
            self.shortcut = nn.________()  # 설명: 입력을 그대로 통과시키는 shortcut 모듈
        # (2) shape가 다르면: Option A(ZeroPad) 또는 Option B(Projection) 중 선택
        else:
            if shortcut == "projection":
                self.shortcut = ________________(in_channels, out_channels, stride=stride)  # 설명: 학습 가능한 1×1 conv shortcut
            elif shortcut == "zero_pad":
                self.shortcut = _____________(in_channels, out_channels, stride=stride)  # 설명: 파라미터 없이 0 채널을 붙이는 shortcut
            else:
                raise ValueError(f"Unknown shortcut mode: {shortcut}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # [CheatSheet §1 Shortcut 연결 방식 | Step 10] shortcut 경로 텐서 준비
        identity = ____._______(__)  # 역할: 메인 경로와 더할 수 있도록 bypass 텐서를 만든다

        # [CheatSheet §0 Residual Function | Step 11] main path: conv1 -> bn1 -> relu
        out = self.____(self.___(self.____(x)))  # 설명: conv1 -> bn1 -> relu 순서로 main path의 첫 번째 conv 블록을 만든다

        # [CheatSheet §0 Residual Function | Step 12] main path: conv2 -> bn2 (relu 없음)
        out = self.___(self.____(out)) # 역할: 합산 직전의 스케일/분포를 안정화한다

        # [CheatSheet §0 Residual Addition (Eq.1) | Step 13] Residual addition
        out = ____ + ________  # 역할: residual branch 결과와 shortcut 결과를 결합한다

        # [CheatSheet §2 Post-activation | Step 14] Post-activation (ResNet v1)
        out = ____.____(out)  # 역할: 합산 결과에 활성화를 적용해 블록 출력을 만든다
        return out


### ✅ Check 3: BasicBlockV1 shape 테스트
(빈칸을 채운 뒤 실행)


In [ ]:
blk = BasicBlockV1(
    in_channels=16,
    out_channels=32,
    stride=2,
    shortcut="projection"
)

x = torch.randn(2, 16, 32, 32)
y = blk(x)

expected = (2, 32, 16, 16)
actual = tuple(y.shape)

if actual != expected:
    raise ValueError(f"shape mismatch: expected {expected}, got {actual}")

print("[OK] BasicBlockV1 shape:", actual)


## 4) (심화) BottleneckV1 ↔ `BottleneckV1`

깊은 ResNet(50/101/152 등)에서 중요한 구조입니다.

- 1×1로 채널을 줄이고 → 3×3에서 계산 → 1×1로 채널을 다시 확장(expansion=4)
- 연산량(특히 3×3)을 줄이면서 깊이를 늘립니다.



In [ ]:

class BottleneckV1(nn.Module):
    """
    Figure 5 (right) bottleneck (ImageNet용에서 주로 사용).
    이 노트북에서는 심화(선택) 섹션으로만 다룹니다.
    """
    expansion = 4  # 최종 출력 채널 = planes * expansion

    def __init__(self, in_channels: int, planes: int, stride: int, shortcut: str = "projection"):
        super().__init__()
        out_channels = planes * self.expansion

        self.conv1 = nn.Conv2d(in_channels, planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.conv3 = nn.Conv2d(planes, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

        # shortcut 선택
        if stride == 1 and in_channels == out_channels:
            self.shortcut = nn.Identity()
        else:
            # Bottleneck에서 zero_pad는 일반적으로 쓰지 않지만, 인터페이스 통일을 위해 허용
            if shortcut == "projection":
                self.shortcut = ShortcutProjection(in_channels, out_channels, stride=stride)
            elif shortcut == "zero_pad":
                self.shortcut = ShortcutZeroPad(in_channels, out_channels, stride=stride)
            else:
                raise ValueError(f"Unknown shortcut mode: {shortcut}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = self.shortcut(x)

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        out = out + identity
        out = self.relu(out)
        return out


### ✅ Check 4: BottleneckV1 shape 테스트 (심화, 선택)


In [ ]:

blk = BottleneckV1(in_channels=64, planes=16, stride=2, shortcut="projection")
x = torch.randn(2, 64, 32, 32)
y = blk(x)
# out_channels = planes*expansion = 16*4=64, stride=2 => 16x16
assert y.shape == (2, 64, 16, 16), f"shape mismatch: {y.shape}"
print("[OK] BottleneckV1 shape:", y.shape)


## 5) ResNetV1 전체 구조 ↔ `ResNetV1.__init__`, `_make_layer`, `forward`

### 핵심 규칙 (CheatSheet 좌표)
- Stage 구조: CheatSheet §4
- Downsampling at Stage Start: CheatSheet §4
- Small Stem (CIFAR): CheatSheet §5
- Global Average Pooling: CheatSheet §6

### 핵심 규칙
- **Stage 구조**: stage 내부에서는 (H,W,채널 패턴)이 일정  
- **Downsampling은 stage 시작에서만**(첫 블록 stride=2)  
- CIFAR는 입력이 작아 **small stem(3×3, stride=1)**로 시작  
- 마지막은 **Global Average Pooling(GAP)** 후 FC


In [ ]:

class ResNetV1(nn.Module):
    def __init__(
        self,
        block: Type[nn.Module],
        layers: List[int],
        num_classes: int,
        stem: str,        # "imagenet" or "cifar"
        shortcut: str,    # "projection" or "zero_pad"
        cifar_base_channels: int = 16,
    ):
        super().__init__()
        self.shortcut = shortcut

        # [CheatSheet §5 Small Stem | Step 15] Stem 구성 (CIFAR vs ImageNet)
        if stem == "imagenet":
            self.in_channels = 64
            self.stem = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            )
            stage_planes = [64, 128, 256, 512]
        elif stem == "cifar":
            # CIFAR는 입력이 작으므로 3×3 conv로 “small stem” (해상도 유지)
            self.in_channels = cifar_base_channels
            self.stem = nn.Sequential(
                nn.Conv2d(3, cifar_base_channels, kernel_size=3, stride=1, padding=1, bias=False),
                nn.BatchNorm2d(cifar_base_channels),
                nn.ReLU(inplace=True),
            )
            # base 채널에서 stage마다 2배씩 증가 (16 → 32 → 64)
            stage_planes = [cifar_base_channels, cifar_base_channels * __, cifar_base_channels * _]
        else:
            raise ValueError("stem must be 'imagenet' or 'cifar'")

        # [CheatSheet §4 Downsampling at Stage Start | Step 16]
        # stage 시작에서만 downsample (CIFAR: layer2, layer3 진입 시 stride=2)
        self.layer1 = self._make_layer(block, stage_planes[0], layers[0], stride=__)
        self.layer2 = self._make_layer(block, stage_planes[1], layers[1], stride=__)
        self.layer3 = self._make_layer(block, stage_planes[2], layers[2], stride=__)

        if stem == "imagenet":
            self.layer4 = self._make_layer(block, stage_planes[3], layers[3], stride=2)
            final_ch = stage_planes[3] * getattr(block, "expansion", 1)
        else:
            self.layer4 = None
            final_ch = stage_planes[2] * getattr(block, "expansion", 1)

        # [CheatSheet §6 Global Average Pooling | Step 17] Global Average Pooling + FC
        self.avgpool = nn.AdaptiveAvgPool2d((__, __))
        self.fc = nn.Linear(final_ch, num_classes)

    def _make_layer(self, block: Type[nn.Module], planes: int, blocks: int, stride: int) -> nn.Sequential:
        """
        stage를 조립합니다.
        - 첫 블록만 stride를 적용해 downsample 가능
        - 이후 블록들은 stride=1로 유지
        """
        layers = []
        expansion = getattr(block, "expansion", 1)

        # Bottleneck은 (in_channels, planes, ...) 입력을 기대 (planes=내부 채널)
        if block is BottleneckV1:
            layers.append(block(self.in_channels, planes, stride=stride, shortcut=self.shortcut))
            self.in_channels = ______ * _________  # 설명: bottleneck 출력 채널로 in_channels를 갱신한다
            for _ in range( __ , blocks):  # 설명: 첫 블록을 이미 추가했으니, 나머지 블록만 반복한다
                layers.append(block(self.in_channels, planes, stride=1, shortcut=self.shortcut))

        # BasicBlock은 (in_channels, out_channels, ...) 입력을 기대
        else:
            out_channels = ______ * _________  # 설명: stage의 출력 채널 수(planes와 expansion의 관계를 생각)
            layers.append(block(self.in_channels, out_channels, stride=stride, shortcut=self.shortcut))
            self.in_channels = ___________  # 설명: 다음 블록 입력 채널을 현재 stage 출력 채널로 갱신한다
            for _ in range(__, blocks):  # 설명: 첫 블록은 이미 넣었으니 남은 블록들만 추가한다
                layers.append(block(self.in_channels, out_channels, stride=1, shortcut=self.shortcut))

        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        if self.layer4 is not None:
            x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


# =========================================================
# 4) Factory (CIFAR: depth = 6n + 2)
# =========================================================


## 6) CIFAR Factory: depth = 6n + 2 ↔ `resnet_cifar`

CIFAR-ResNet에서 depth는 **6n+2** 형태여야 합니다.

- conv는 블록당 2개  
- stage가 3개이고 각 stage에 n 블록  
- 그래서 conv 층 수가 2×(3×n)=6n, 여기에 stem conv 1개 + classifier 1개를 더해 6n+2

In [ ]:

def resnet_cifar(depth: int, num_classes: int = 10, shortcut: str = "zero_pad"):
    """
    CIFAR-ResNet (논문 §4.2):
    - depth = 6n + 2 형태여야 함 (예: 20/32/44/56/110/...)
    - 각 stage의 블록 개수는 모두 n
    """
    # (1) depth 규칙 검사
    if (_____ - __) % __ != __:  # 설명: depth가 6n+2 형태가 아니면 예외를 발생시킨다
        raise ValueError("CIFAR depth must be 6n+2")

    # (2) n 역산
    n = (_____ - __) // __  # 설명: depth에서 n을 역으로 계산한다 (정수로 떨어져야 함)

    # (3) stage별 블록 개수: [n, n, n]
    return ResNetV1(BasicBlockV1, [__, __, __], num_classes, stem="cifar", shortcut=shortcut)


## Appendix: ImageNet용 ResNet 팩토리 함수

CIFAR-10 실험은 논문 §4.2의 **단순화된 구조(6n+2, 32×32 입력, 3 stages)**를 쓰지만,
ResNet의 “근본”은 ImageNet 설정(224×224 입력, 4 stages, 18/34는 BasicBlock, 50+는 Bottleneck)입니다.

- 이 섹션은 **학습 필수는 아니고**, 구조 감 잡기/확장 실습을 위해 **참고로 포함**했습니다.
- 빈칸은 없고, 그대로 실행 가능한 유틸 함수입니다.


In [ ]:

# ResNet 모델 생성 함수 (ImageNet 용) — 참고용
def resnet_imagenet(depth: int, num_classes: int = 1000, shortcut: str = "projection"):
    """
    ImageNet 설정의 ResNet 팩토리 함수.
    - 18/34: BasicBlock
    - 50/101/152: Bottleneck
    - shortcut: "projection" 또는 "zero_pad"(실험/학습용). 실전 ImageNet은 보통 projection을 사용합니다.
    """
    if depth == 18:
        return ResNetV1(BasicBlockV1, [2, 2, 2, 2], num_classes, stem="imagenet", shortcut=shortcut)
    if depth == 34:
        return ResNetV1(BasicBlockV1, [3, 4, 6, 3], num_classes, stem="imagenet", shortcut=shortcut)
    if depth == 50:
        return ResNetV1(BottleneckV1, [3, 4, 6, 3], num_classes, stem="imagenet", shortcut=shortcut)
    if depth == 101:
        return ResNetV1(BottleneckV1, [3, 4, 23, 3], num_classes, stem="imagenet", shortcut=shortcut)
    if depth == 152:
        return ResNetV1(BottleneckV1, [3, 8, 36, 3], num_classes, stem="imagenet", shortcut=shortcut)
    raise ValueError("Unsupported depth for ImageNet ResNet")


### ✅ Check 5: ResNet forward / depth rule 테스트


In [ ]:

# depth rule
try:
    resnet_cifar(depth=20)  # 20=6*3+2 OK
except Exception as e:
    raise AssertionError(f"depth=20 should be valid, but got: {e}")

try:
    resnet_cifar(depth=21)  # invalid
    raise AssertionError("depth=21 should be invalid but passed")
except ValueError:
    print("[OK] invalid depth correctly raises ValueError")

# forward shape
m = resnet_cifar(depth=56, num_classes=10, shortcut="zero_pad")
x = torch.randn(4, 3, 32, 32)
with torch.no_grad():
    out = m(x)

expected = (4, 10)
actual = tuple(out.shape)
if actual != expected:
    raise ValueError(f"output shape mismatch: expected {expected}, got {actual}")

print("[OK] ResNet forward:", out.shape)



## 7) Train/Eval 루프 ↔ `train_one_epoch`, `evaluate`

### 핵심 포인트
- 학습: `model.train()` + gradient 필요  
- 평가: `model.eval()` + `torch.no_grad()`로 gradient off

In [ ]:

# =========================================================
# 7) Train/Eval 유틸리티 (빈칸 없음)
# - 아래 실습 섹션(§8)에서도 그대로 활용할 수 있습니다.
# =========================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T


def build_transforms():
    """논문 §4.2 CIFAR-10 augmentation (기본 설정)."""
    train_tf = T.Compose([
        T.RandomCrop(32, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
    ])
    test_tf = T.Compose([T.ToTensor()])
    return train_tf, test_tf


def build_dataloaders(batch_size: int = 128, num_workers: int = 4):
    train_tf, test_tf = build_transforms()
    trainset = torchvision.datasets.CIFAR10("./data", train=True, download=True, transform=train_tf)
    testset = torchvision.datasets.CIFAR10("./data", train=False, download=True, transform=test_tf)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return trainloader, testloader


def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    return total_loss / max(total, 1), correct / max(total, 1)


def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)

            logits = model(x)
            loss = criterion(logits, y)

            total_loss += loss.item() * x.size(0)
            pred = logits.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)

    return total_loss / max(total, 1), correct / max(total, 1)


### ✅ Check 6: (데이터 다운로드 없이) 더미 배치로 train/eval 루프 스모크 테스트


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
model = resnet_cifar(depth=20, num_classes=10, shortcut="projection").to(device)

# dummy dataset
x = torch.randn(64, 3, 32, 32)
y = torch.randint(0, 10, (64,))
loader = DataLoader(TensorDataset(x, y), batch_size=16, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# train/eval should run
train_loss, train_acc = train_one_epoch(model, loader, criterion, optimizer, device)
test_loss, test_acc = evaluate(model, loader, criterion, device)

# assert -> if/raise
if not (isinstance(train_loss, float) and isinstance(train_acc, float)):
    raise TypeError(
        f"train outputs must be float, got train_loss={type(train_loss)}, train_acc={type(train_acc)}"
    )

if not (isinstance(test_loss, float) and isinstance(test_acc, float)):
    raise TypeError(
        f"eval outputs must be float, got test_loss={type(test_loss)}, test_acc={type(test_acc)}"
    )

print("[OK] smoke test:", train_loss, train_acc, test_loss, test_acc)


## 8) 실습: CIFAR-10 학습을 실제로 돌려보기 (논문 설정 정렬)

아래 코드는 **논문 §4.2 설정에 맞춘 CIFAR-10 학습 스켈레톤**입니다.

- 이 섹션은 **빈칸이 없습니다.**
- `epochs=164`는 시간이 오래 걸릴 수 있으니, 처음에는 `epochs=1~3`으로 줄여서 동작 확인을 권장합니다.
- 노트북에서는 `resnet_cifar`가 이미 위에서 정의되어 있으므로, `from model import ...` 같은 외부 import가 필요 없습니다.


In [ ]:

# train_cifar10.py (paper-aligned CIFAR-10 training skeleton) — notebook friendly

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T


def main(epochs: int = 164, batch_size: int = 128, num_workers: int = 4):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Examples: 20/32/44/56/110... where depth = 6n+2
    model = resnet_cifar(depth=56, num_classes=10, shortcut="zero_pad").to(device)

    # Data augmentation described in the paper (§4.2)
    train_tf = T.Compose([
        T.RandomCrop(32, padding=4),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
    ])
    test_tf = T.Compose([T.ToTensor()])

    trainset = torchvision.datasets.CIFAR10("./data", train=True, download=True, transform=train_tf)
    testset = torchvision.datasets.CIFAR10("./data", train=False, download=True, transform=test_tf)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

    # Paper: divide by 10 at 32k and 48k iters, stop at 64k iters.
    # 50k/128 ≈ 391 iters/epoch → 32k≈82ep, 48k≈123ep, 64k≈164ep
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[82, 123], gamma=0.1)

    for epoch in range(epochs):
        model.train()
        for x, y in trainloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total, test_loss = 0, 0, 0.0
        with torch.no_grad():
            for x, y in testloader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                test_loss += criterion(logits, y).item() * x.size(0)
                pred = logits.argmax(dim=1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        scheduler.step()
        print(f"epoch={epoch:03d} acc={100.0*correct/total:.2f} loss={test_loss/total:.4f}")


# ✅ 실행 예시 (처음엔 epochs를 줄여서!)
# main(epochs=3)


## (정답 공개) — 마지막에만 확인하세요


In [ ]:

# ===========================
# ANSWERS (예시) — 마지막에 확인하세요
# ===========================

# --- ShortcutZeroPad ---
# (H,W) downsample: x = x[:, :, ::self.stride, ::self.stride]
# pad_channels = self.out_channels - self.in_channels
# zeros shape: (x.size(0), pad_channels, x.size(2), x.size(3))
# torch.cat dim=1

# --- BasicBlockV1 ---
# nn.Identity()
# ShortcutProjection / ShortcutZeroPad
# identity = self.shortcut(x)
# relu 적용: self.relu(out)
# bn2 적용: self.bn2(out)
# residual add: out = out + identity

# --- ResNetV1._make_layer ---
# bottleneck in_channels update: planes * expansion
# for _ in range(1, blocks):
# basic out_channels = planes * expansion
# self.in_channels = out_channels

# --- resnet_cifar ---
# if (depth - 2) % 6 != 0: raise
# n = (depth - 2) // 6
# layers = [n, n, n]
